<a href="https://colab.research.google.com/github/doquyenduong/deep_learning_vision/blob/main/Twitter_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text generation with an RNN

This code is to generate text using a character-based RNN. The dataset is the combination of twitter comments. Given a sequence of characters from this data, train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.

While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words, but consider:

* The model is character-based. When training started, the model did not know how to spell an English word, or that words were even a unit of text.

* The structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.

* As demonstrated below, the model is trained on small batches of text (100 characters each), and is still able to generate a longer sequence of text with coherent structure.

## Setup

### Import TensorFlow and other libraries

In [6]:
import tensorflow as tf
from tensorflow.keras.utils import get_file
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import get_file
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Input, Embedding, Dropout, Activation
import numpy as np
import random
import sys
import io
import string

import numpy as np
import os
import time

### Download the song lyrics dataset

Change the following line to run this code on data from Kaggle: https://www.kaggle.com/datasets/pawankumar97/us-twitter-news-blogs-datasets?select=en_US.twitter.txt

In [1]:
path_to_file = '/content/en_US.twitter.txt'

### Read the data

First, look in the text:

In [2]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 21981895 characters


In [3]:
# Take a look at the first 250 characters in text
print(text[:250])

How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way too long.
When you meet someone special... you'll know. Your heart will beat more rapidly and you'll smile for no reason.
they've decided its more f


In [4]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

958 unique characters


## Process the text

### Vectorize the text

Before training, we need to convert the strings to a numerical representation. 

The `tf.keras.layers.StringLookup` layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [7]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Now create the `tf.keras.layers.StringLookup` layer:

In [8]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

It converts from tokens to character IDs:

In [9]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[69, 70, 71, 72, 73, 74, 75], [92, 93, 94]]>

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this we can use `tf.keras.layers.StringLookup(..., invert=True)`.  

Note: Here instead of passing the original vocabulary generated with `sorted(set(text))` use the `get_vocabulary()` method of the `tf.keras.layers.StringLookup` layer so that the `[UNK]` tokens is set the same way.

In [10]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

This layer recovers the characters from the vectors of IDs, and returns them as a `tf.RaggedTensor` of characters:

In [11]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

We can `tf.strings.reduce_join` to join the characters back into strings. 

In [12]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [13]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [14]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(21981895,), dtype=int64, numpy=array([44, 83, 91, ..., 86,  4, 74])>

In [15]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [16]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

H
o
w
 
a
r
e
 
y
o


In [17]:
seq_length = 100


The `batch` method lets we easily convert these individual characters to sequences of the desired size.

In [18]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'H' b'o' b'w' b' ' b'a' b'r' b'e' b' ' b'y' b'o' b'u' b'?' b' ' b'B'
 b't' b'w' b' ' b't' b'h' b'a' b'n' b'k' b's' b' ' b'f' b'o' b'r' b' '
 b't' b'h' b'e' b' ' b'R' b'T' b'.' b' ' b'Y' b'o' b'u' b' ' b'g' b'o'
 b'n' b'n' b'a' b' ' b'b' b'e' b' ' b'i' b'n' b' ' b'D' b'C' b' ' b'a'
 b'n' b'y' b't' b'i' b'm' b'e' b' ' b's' b'o' b'o' b'n' b'?' b' ' b'L'
 b'o' b'v' b'e' b' ' b't' b'o' b' ' b's' b'e' b'e' b' ' b'y' b'o' b'u'
 b'.' b' ' b'B' b'e' b'e' b'n' b' ' b'w' b'a' b'y' b',' b' ' b'w' b'a'
 b'y' b' ' b't'], shape=(101,), dtype=string)


It's easier to see what this is doing if we join the tokens back into strings:

In [19]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way t'
b"oo long.\r\nWhen you meet someone special... you'll know. Your heart will beat more rapidly and you'll "
b"smile for no reason.\r\nthey've decided its more fun if I don't.\r\nSo Tired D; Played Lazer Tag & Ran A "
b'LOT D; Ughh Going To Sleep Like In 5 Minutes ;)\r\nWords from a complete stranger! Made my birthday eve'
b'n better :)\r\nFirst Cubs game ever! Wrigley field is gorgeous. This is perfect. Go Cubs Go!\r\ni no! i g'


For training we'll need a dataset of `(input, label)` pairs. Where `input` and 
`label` are sequences. At each time step the input is the current character and the label is the next character. 

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [20]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [21]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [22]:
dataset = sequences.map(split_input_target)

In [23]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way '
Target: b'ow are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way t'


### Create training batches

we used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [24]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## Build The Model

This section defines the model as a `keras.Model` subclass (For details see [Making new Layers and Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models)). 

This model has three layers:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (we can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [25]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [26]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [27]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/text_generation_training.png?raw=1)

Note: For training we could use a `keras.Sequential` model here. To  generate text later we'll need to manage the RNN's internal state. It's simpler to include the state input and output options upfront, than it is to rearrange the model architecture later. For more details see the [Keras RNN guide](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse).

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [28]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 959) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [29]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  245504    
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  982975    
                                                                 
Total params: 5,166,783
Trainable params: 5,166,783
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [30]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [31]:
sampled_indices

array([771, 741, 280, 882, 139, 416, 562, 425, 768, 784, 766, 874, 938,
       337, 463, 815, 116, 703, 259, 388, 422, 525, 948,  17,  69, 702,
       821, 217, 279, 563, 157,   4, 444, 843,  82,  96,  11, 278, 585,
       933, 459, 348, 704, 406,  26, 863,  29, 950, 250, 363, 830, 403,
       313, 442, 306, 378, 101, 340, 272,  38, 280, 575, 462,  86, 466,
       538, 144, 798, 505, 329, 116, 434, 619, 462, 924, 441, 682, 429,
       743, 100, 580, 223, 297, 264, 696, 740, 721, 308, 338, 565,  26,
       318, 543, 947, 941, 356, 508, 857, 275, 788])

Decode these to see the text predicted by this untrained model:

In [32]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'sick" means "awesome" and "I love you" doesn\'t mean anything.\r\nWhip it.\r\nI thnk I want some cereal\r\n'

Next Char Predictions:
 b"\xf0\x9f\x8f\x80\xf0\x9f\x8d\xb0\xe2\x88\xa9\xf0\x9f\x92\xbc\xc3\x96\xe3\x81\xaf\xee\x80\xb2\xe3\x82\x8b\xf0\x9f\x8e\xb6\xf0\x9f\x90\x96\xf0\x9f\x8e\xb0\xf0\x9f\x92\xa3\xf0\x9f\x99\x86\xe2\x99\x81\xe4\xb8\x87\xf0\x9f\x91\x8d\xc2\xab\xf0\x9f\x87\xa7\xe2\x86\x90\xe3\x80\xbd\xe3\x82\x82\xe8\xb2\xac\xf0\x9f\x9a\x99-a\xf0\x9f\x86\x94\xf0\x9f\x91\x95\xd6\xb6\xe2\x88\x9e\xee\x80\xb3\xc3\xad \xe3\x83\x88\xf0\x9f\x91\xben|'\xe2\x88\x9a\xee\x84\x85\xf0\x9f\x98\xb1\xe3\x85\x8b\xe2\x99\xac\xf0\x9f\x87\xac\xe3\x81\xa16\xf0\x9f\x92\x959\xf0\x9f\x9a\xa4\xe2\x80\xb9\xe2\x9c\x97\xf0\x9f\x91\xa7\xe3\x81\x9c\xe2\x98\x80\xe3\x83\x85\xe2\x97\x84\xe2\x9d\xa7\xc2\x94\xe2\x99\xa0\xe2\x87\xa2B\xe2\x88\xa9\xee\x81\x8c\xe4\xb8\x83r\xe4\xba\xba\xeb\xa6\xbd\xc3\x9f\xf0\x9f\x90\xb4\xe6\xa8\xaa\xe2\x98\xa0\xc2\xab\xe3\x82\xaf\xee\x8c\xa6\xe4\xb8\x83\xf0\x9f\x98\xa0\xe3\x83

## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because wer model returns logits, we need to set the `from_logits` flag.


In [33]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [34]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 959)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(6.865557, shape=(), dtype=float32)


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this we can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [35]:
tf.exp(example_batch_mean_loss).numpy()

958.6799

Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [36]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [37]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [38]:
EPOCHS = 20

In [39]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
3400/3400 [==============================] - 157s 45ms/step - loss: 1.8024
Epoch 2/20
3400/3400 [==============================] - 154s 45ms/step - loss: 1.5015
Epoch 3/20
3400/3400 [==============================] - 154s 45ms/step - loss: 1.4526
Epoch 4/20
3400/3400 [==============================] - 153s 45ms/step - loss: 1.4276
Epoch 5/20
3400/3400 [==============================] - 153s 45ms/step - loss: 1.4122
Epoch 6/20
3400/3400 [==============================] - 153s 45ms/step - loss: 1.4018
Epoch 7/20
3400/3400 [==============================] - 153s 45ms/step - loss: 1.3947
Epoch 8/20
3400/3400 [==============================] - 154s 45ms/step - loss: 1.3897
Epoch 9/20
3400/3400 [==============================] - 156s 45ms/step - loss: 1.3862
Epoch 10/20
3400/3400 [==============================] - 155s 45ms/step - loss: 1.3836
Epoch 11/20
3400/3400 [==============================] - 154s 45ms/step - loss: 1.3828
Epoch 12/20
3400/3400 [=============================

## Generate text

The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as we execute it.

![To generate text the model's output is fed back to the input](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/text_generation_sampling.png?raw=1)

Each time we call the model we pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.


The following makes a single step prediction:

In [40]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [41]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Run it in a loop to generate some text. Looking at the generated text, we'll see the model knows when to capitalize, make paragraphs and imitates a twitter-like vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [51]:
start = time.time()
states = None
next_char = tf.constant(['congratulation:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

congratulation: Any interesting. MLB colemals are grandsquibed in seculd bound balancial.
The fact that and balance and lovely shouldn't like nothing to appeal to teen to me back soon to do
Tonight I cannot waited 3 hours away for 3 rules...
Kacheen practice=restroom/like something like to yumm. <3 no way!
Not gonna make UK gosh you up the truth soon.
Morning is Daniel Cirlin Juna #Mel'buelles Twitterverse!" maybe you organized.
Thanks for Twisters I may how about brought to live
Not that I can't DM for Vernnyn new projects and Pak. We loved bad for posting . Thank you for some?
I'm at work amazing....are u in personal.
happy gran v. Col
I'm sure money makes you spice!
The fear from Romney is grows unlecting them far from anheleans just to take the new GiangFish team butter we still play?
Tell someone also actually searchingly reached.
It is chillen-over. Maybe for practice is... Where do sushi stones and albu?
two weeks" than a mile twitter I want to bed?
Alright figure out that ...
 

In [54]:
start = time.time()
states = None
next_char = tf.constant(['money:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

money:/ I hate the in a row. Shin all things just let among had a computer.
wow! I can't believe I'm tweeting with your tweety Livi.
#BlackFaphonyfriend tell whitney devices in the space '00 yet in* He
Go da problems ;-)
so when website >;)<<<<
Sowhere - I'm in classes deadly.
HI AND TODAY we beat the WellinGHAC after ChamPage! Blessings that are you come with me?? And I'm thinking make his glasses today thank you.
Thanks For Disney! It always Talented Thanks Crispy Jarreling. RT !!
Let
We NOT will be spating practice there. Thanks for the follow!
what is goin wood??
haha that was powerful?
Agreed. You're rich Christmas Shouldn't.
Thx for looking for a great education show and good . Lil Bit girl - Tweethney Drive is so fun stuins at Mondar full of stages plantrone
just won, not make tax demands shootly.
UNC. stay!
We're sure in honored boory really bad for cake bandanas TV in Nelsonic?
Myspbins sby" smallers blussions in the corner... & too main for all bands, VERY. observi 

________

The easiest thing we can do to improve the results is to train it for longer (try `EPOCHS = 30`).

we can also experiment with a different start string, try adding another RNN layer to improve the model's accuracy, or adjust the temperature parameter to generate more or less random predictions.

If we want the model to generate text *faster* the easiest thing we can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above. 

In [43]:
start = time.time()
states = None
next_char = tf.constant(['we:', 'we:', 'we:', 'we:', 'we:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b'you:) Looks to blame on redness maybe yolumes, fast I\'ve seen your Stealer.\r\nI would have liked to pass any time fantasy without awesome! Drinkin Would Like I\r\nWhat\'se bad fuck you happies :)\r\nAnyone else uplife this ? RT\r\nTheZon Ground temparious ages are black and I had to wrise tonight. Suggested Pike none-scall coming above tonight\'s stuff? Make the most of the best driving Many. I think I planter that cold down.\r\nWe lot of students!" What a wonderful mobery game class\r\nRT : Chem value. Someone who says I think I say... You must know an endrepeloasle every answer.\r\nWhats that faith in my weekend ? I use to renove your 12 opinion after a year. they check them out over my new place!\r\nTravels. Sweet Blog el.\r\nThx Sully and The Onyonen day!!\r\nSimplies so WON\'T MEST FOR THE MENO.\r\nWHAT? whats up with Dogger Tuesday fans platinum!\r\nIm out numbers \xf0\x9f\x92\x8d I surd listening to em on VHfollow // I bet the college car is the perfect idea of s

## Export the generator

This single-step model can easily be [saved and restored](https://www.tensorflow.org/guide/saved_model), allowing we to use it anywhere a `tf.saved_model` is accepted.

In [44]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


In [45]:
states = None
next_char = tf.constant(['we:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

you: Now, that's a year.
I remember n I just voluented work halftime today and I'm set a cuddles warped


## Advanced: Customized Training

The above training procedure is simple, but does not give we much control.
It uses teacher-forcing which prevents bad predictions from being fed back to the model, so the model never learns to recover from mistakes.

So now that we've seen how to run the model manually next we'll implement the training loop. This gives a starting point if, for example, we want to implement _curriculum  learning_ to help stabilize the model's open-loop output.

The most important part of a custom training loop is the train step function.

Use `tf.GradientTape` to track the gradients. we can learn more about this approach by reading the [eager execution guide](https://www.tensorflow.org/guide/eager).

The basic procedure is:

1. Execute the model and calculate the loss under a `tf.GradientTape`.
2. Calculate the updates and apply them to the model using the optimizer.

In [46]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

The above implementation of the `train_step` method follows [Keras' `train_step` conventions](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit). This is optional, but it allows we to change the behavior of the train step and still use keras' `Model.compile` and `Model.fit` methods.

In [47]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [48]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [49]:
model.fit(dataset, epochs=1)

3400/3400 [==============================] - 155s 45ms/step - loss: 1.8002


Or if we need more control, we can write wer own complete custom training loop:

In [50]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 1.4918
Epoch 1 Batch 50 Loss 1.5765
Epoch 1 Batch 100 Loss 1.5112
Epoch 1 Batch 150 Loss 1.5684
Epoch 1 Batch 200 Loss 1.5199
Epoch 1 Batch 250 Loss 1.5322
Epoch 1 Batch 300 Loss 1.5449
Epoch 1 Batch 350 Loss 1.4897
Epoch 1 Batch 400 Loss 1.5071
Epoch 1 Batch 450 Loss 1.5119
Epoch 1 Batch 500 Loss 1.5389
Epoch 1 Batch 550 Loss 1.5955
Epoch 1 Batch 600 Loss 1.4846
Epoch 1 Batch 650 Loss 1.5384
Epoch 1 Batch 700 Loss 1.5160
Epoch 1 Batch 750 Loss 1.5594
Epoch 1 Batch 800 Loss 1.4641
Epoch 1 Batch 850 Loss 1.5025
Epoch 1 Batch 900 Loss 1.5336
Epoch 1 Batch 950 Loss 1.5445
Epoch 1 Batch 1000 Loss 1.5480
Epoch 1 Batch 1050 Loss 1.5508
Epoch 1 Batch 1100 Loss 1.4872
Epoch 1 Batch 1150 Loss 1.5170
Epoch 1 Batch 1200 Loss 1.5191
Epoch 1 Batch 1250 Loss 1.5473
Epoch 1 Batch 1300 Loss 1.4708
Epoch 1 Batch 1350 Loss 1.5124
Epoch 1 Batch 1400 Loss 1.5291
Epoch 1 Batch 1450 Loss 1.4794
Epoch 1 Batch 1500 Loss 1.5036
Epoch 1 Batch 1550 Loss 1.4411
Epoch 1 Batch 1600 Loss 1.4659


KeyboardInterrupt: ignored